In [1]:
# Third-party libraries
from bs4 import BeautifulSoup
import bs4 as bs
import urllib.request
import requests
import pandas as pd
import numpy as np
import json
from functools import reduce

In [2]:
url = "https://www.forsyth.k12.ga.us/Page/52982"

In [3]:
# Make a GET request to fetch the raw HTML content
html_content = requests.get(url).text

In [4]:
# Parse the html content
soup = BeautifulSoup(html_content, "html.parser")
# print(soup.prettify()) # print the parsed data of html

In [5]:
covid_links = {}
for link in soup.find_all("a"):
    if str(link.text).lower().find('week ') != -1:
        covid_links[str(link.text)] = str(link['href']).replace('s','',1)
    else:
        pass

print(covid_links)

{'Week 1': 'http://www.forsyth.k12.ga.us/Page/53161', 'Week 2': 'http://www.forsyth.k12.ga.us/Page/53166', 'Week 3': 'http://www.forsyth.k12.ga.us/Page/53167'}


In [6]:
covid_url = covid_links['Week 2']
covid_url

'http://www.forsyth.k12.ga.us/Page/53166'

In [7]:
source = requests.get('http://www.forsyth.k12.ga.us/Page/53166').text
soup_wk1 = bs.BeautifulSoup(source,'html.parser')

In [8]:
token = soup_wk1.find_all('input', id=lambda x: x and x.startswith('hidData'))

In [9]:
info = [x.get('value') for x in token][0]

In [10]:
replace_list = ['<p>','</p>','<p class=\\"\\', '>']
for t in replace_list:
    info = info.replace(t, '')
info = info.replace('""', '"')
print(info)

[["School Name","Total Staff & Student","8/17/20","8/18/20","8/19/20","8/20/20","8/21/20"],["ELEMENTARY",null,null,null,null,null,null],["Big Creek ES","706","0","0","0","0","0"],["Brandywine ES","1,357","0","0","0","0","1"],["Brookwood ES","1,078","0","0","0","0","0"],["Chattahoochee ES","850","0","2","0","0","0"],["Chestatee ES","1,381","0","0","1","0","0"],["Coal Mountain ES","821","0","0","2","0","0"],["Cumming ES","1,055","0","0","0","0","0"],["Daves Creek ES","1,285","0","0","0","0","0"],["Haw Creek ES","1,131","0","0","0","0","0"],["Johns Creek ES","1,103","0","0","0","0","0"],["Kelly Mill ES","1,257","0","0","0","0","0"],["Mashburn ES","781","0","0","0","0","0"],["Matt ES","1,125","0","0","0","0","0"],["Midway ES","961","0","0","0","0","0"],["Poole's Mill ES","807","0","0","0","1","0"],["Sawnee ES","1,459","0","0","0","0","0"],["Settles Bridge ES","1,099","0","0","0","0","0"],["Sharon ES","999","0","0","0","0","0"],["Shiloh Point ES","1,452","0","0","0","0","1"],["Silver City E

In [11]:
info_json = json.loads(info)

In [12]:
headers = info_json.pop(0)
df = pd.DataFrame(info_json, columns=headers)
df.head()

,School Name,Total Staff & Student,8/17/20,8/18/20,8/19/20,8/20/20,8/21/20
0,ELEMENTARY,None,None,None,None,None,None
1,Big Creek ES,706,0,0,0,0,0
2,Brandywine ES,"1,357",0,0,0,0,1
3,Brookwood ES,"1,078",0,0,0,0,0
4,Chattahoochee ES,850,0,2,0,0,0


In [13]:
columns = df.columns
columns

Index(['School Name', 'Total Staff & Student', '8/17/20', '8/18/20', '8/19/20',
       '8/20/20', '8/21/20'],
      dtype='object')

In [14]:
for col in columns[1:]:
    df[col] = pd.to_numeric(df[col].str.replace(',',''), errors='coerce')

In [15]:
df.head()

,School Name,Total Staff & Student,8/17/20,8/18/20,8/19/20,8/20/20,8/21/20
0,ELEMENTARY,NaN,NaN,NaN,NaN,NaN,NaN
1,Big Creek ES,706.0,0.0,0.0,0.0,0.0,0.0
2,Brandywine ES,1357.0,0.0,0.0,0.0,0.0,1.0
3,Brookwood ES,1078.0,0.0,0.0,0.0,0.0,0.0
4,Chattahoochee ES,850.0,0.0,2.0,0.0,0.0,0.0


In [16]:
df.replace(to_replace=['None'], value=np.nan, inplace=True)
df.replace(to_replace=[','], value=[''], inplace=True)
df.dropna(subset=['Total Staff & Student'], inplace=True)

In [17]:
df.head()

,School Name,Total Staff & Student,8/17/20,8/18/20,8/19/20,8/20/20,8/21/20
1,Big Creek ES,706.0,0.0,0.0,0.0,0.0,0.0
2,Brandywine ES,1357.0,0.0,0.0,0.0,0.0,1.0
3,Brookwood ES,1078.0,0.0,0.0,0.0,0.0,0.0
4,Chattahoochee ES,850.0,0.0,2.0,0.0,0.0,0.0
5,Chestatee ES,1381.0,0.0,0.0,1.0,0.0,0.0


In [18]:
split_names = df['School Name'].str.rsplit(' ', 1, expand=True)

In [19]:
split_names

,0,1
1,Big Creek,ES
2,Brandywine,ES
3,Brookwood,ES
4,Chattahoochee,ES
5,Chestatee,ES
6,Coal Mountain,ES
7,Cumming,ES
8,Daves Creek,ES
9,Haw Creek,ES
10,Johns Creek,ES


In [20]:
df['school'] = split_names[0]
df['type'] = split_names[1]

In [21]:
df.drop(['School Name'], axis=1, inplace=True)
cols = df.columns.tolist()
cols = cols[-2:] + cols[:-2]
df = df[cols]
df.head()

,school,type,Total Staff & Student,8/17/20,8/18/20,8/19/20,8/20/20,8/21/20
1,Big Creek,ES,706.0,0.0,0.0,0.0,0.0,0.0
2,Brandywine,ES,1357.0,0.0,0.0,0.0,0.0,1.0
3,Brookwood,ES,1078.0,0.0,0.0,0.0,0.0,0.0
4,Chattahoochee,ES,850.0,0.0,2.0,0.0,0.0,0.0
5,Chestatee,ES,1381.0,0.0,0.0,1.0,0.0,0.0


In [22]:
def get_urls(url):
    # Make a GET request to fetch the raw HTML content
    html_content = requests.get(url).text
    # Parse the html content
    soup = BeautifulSoup(html_content, "html.parser")
    
    # Getting weekly number lists
    covid_links = []
    for link in soup.find_all("a"):
        if str(link.text).lower().find('week ') != -1:
            # replacing https with http and getting link url
            covid_links.append(str(link['href']).replace('s','',1))
        else:
            pass
    
    return covid_links

In [23]:
def get_weekly_info(url):   
    source = requests.get(url).text
    soup = bs.BeautifulSoup(source,'html.parser')
    
    # finding hidData element
    token = soup.find_all('input', id=lambda x: x and x.startswith('hidData'))
    # getting 'value' from inside tag
    info = [x.get('value') for x in token][0]
    
    return info

In [24]:
info = get_weekly_info("http://www.forsyth.k12.ga.us/Page/53161")

In [25]:
def info_dataframe(info):
    
    # Removing html characters
    replace_list = ['<p>','</p>','<p class=\\"\\', '>']
    for t in replace_list:
        info = info.replace(t, '')
        
    # Removing extra ""
    info = info.replace('""', '"')
    
    # Replacing data for Alliance Academy
    info = info.replace('Innovation', 'Innovation HS')
    
    # Reading info as json string
    info_json = json.loads(info)
    
    # Removing headers
    headers = info_json.pop(0)
    
    # Reading into dataframe
    df = pd.DataFrame(info_json, columns=headers)
    
    return df

In [26]:
def clean_df(df):
    # Removing , fron numbers and converting to numeric values
    columns = df.columns
    for col in columns[1:]:
        df[col] = pd.to_numeric(df[col].str.replace(',',''), errors='coerce')
    
    # Replacing none values with nulls
    df.replace(to_replace=['None'], value=np.nan, inplace=True)
    # Dropping rows with nulls
    df.dropna(subset=['Total Staff & Student'], inplace=True)
    
    # Splitting school name into name and category
    split_names = df['School Name'].str.rsplit(' ', 1, expand=True)
    # Creating columns for name and category
    df['school'] = split_names[0]
    df['type'] = split_names[1]
    
    # Final cleaning and ordering of columns
    df.drop(['School Name'], axis=1, inplace=True)
    cols = df.columns.tolist()
    cols = cols[-2:] + cols[:-2]
    df = df[cols]
    
    return df

In [27]:
def get_weekly_dataframes(master_url):
    
    # Getting weekly links
    url_links = get_urls(master_url)
    
    # Getting dataframe for each week
    df_list = []
    for url in url_links:
        info = get_weekly_info(url)
        df = info_dataframe(info)
        df = clean_df(df)
        df_list.append(df)
        
    return df_list

In [28]:
def merge_dataframes(df_list):
    df_final = reduce(lambda x, y: pd.merge(x, y, on = ['school', 'type', 'Total Staff & Student']), df_list)
    print(df_final.head())
    print(df_final.shape)
    
    return df_final

In [29]:
def get_covid_data(url):
    df_list = get_weekly_dataframes(url)
    df_final = merge_dataframes(df_list)
    
    # Columns to sum
    df_columns = df_final.columns
    df_columns = df_columns[3:]
    
    # Adding sum column
    df_final['total_positive'] = df_final[df_columns].sum(axis=1)
    df_final['percent_positive'] = (df_final['total_positive'] / df_final['Total Staff & Student']) * 100
    
    # Cleaning type data
    df_final['type'] = df_final['type'].str.strip()
    
    # Dropping TOTAL row
    df_final = df_final[df_final['school'] != 'TOTAL']
    
    return df_final

In [30]:
df_final = get_covid_data("https://www.forsyth.k12.ga.us/Page/52982")

KeyError: ['Total Staff & Student']

In [31]:
df_final.head(50)

NameError: name 'df_final' is not defined

In [ ]:
total_positive = df_final['total_positive'].sum()
total_positive

In [ ]:
columns = df_final.columns
daily_columns = columns[3:-3]

daily_results = df_final[daily_columns]
daily_results

daily_totals = daily_results.sum()
daily_totals

In [ ]:
daily_totals.index[-1]

In [ ]:
import plotly.graph_objects as go

fig_indicator = go.Figure()

fig_indicator.add_trace(go.Indicator(
    mode = "number+delta",
    value = int(daily_totals[-1]),
    title = {"text": f"Daily Cases ({daily_totals.index[-1]})<br><span style='font-size:0.8em;color:gray'>Increase from {daily_totals.index[-2]}</span>"},
    delta = {'reference': int(daily_totals[-1]), 'relative': True, 'position' : "bottom"}))

fig_indicator.show()

In [ ]:
import plotly.graph_objects as go

fig = go.Figure()

fig.add_trace(go.Indicator(
    mode = "number+delta",
    value = int(daily_totals.sum()),
    title = {"text": f"Total Cases ({daily_totals.index[-1]})<br><span style='font-size:0.8em;color:gray'>Increase from {daily_totals.index[-2]}</span>"},
    delta = {'reference': int(daily_totals[:-1].sum()), 'relative': True, 'position' : "bottom"}))

fig.show()

In [ ]:
df_final['type'] = df_final['type'].str.strip()
df_grouped = df_final.groupby('type')

In [ ]:
df_grouped_cols = df_grouped[daily_columns].sum()
df_grouped_cols
df_grouped_yesterday = df_grouped_cols.iloc[:,:-1]
df_grouped_yesterday
# df_grouped_cols

In [ ]:
type_grouped = df_grouped_cols.sum(axis=1)
type_grouped
type_grouped_yesterday = df_grouped_yesterday.sum(axis=1)
type_grouped_yesterday

In [ ]:
import plotly.graph_objects as go

fig = go.Figure()

fig.add_trace(go.Indicator(
    mode = "number+delta",
    value = int(type_grouped[0]),
#     title = {"text": f"Elementary School Total"},
    domain = {'x': [0, 0.4], 'y': [0, 0.25]},
    align = "center",
    delta = {'reference': int(type_grouped_yesterday[0]), 'relative': True, 'position' : "right"}))

fig.add_trace(go.Indicator(
    mode = "number+delta",
    value = int(type_grouped[2]),
#     title = {"text": f"Middle School Total"},
    domain = {'x': [0, 0.4], 'y': [0.36, 0.62]},
    align = "center",
    delta = {'reference': int(type_grouped_yesterday[2]), 'relative': True, 'position' : "right"}))

fig.add_trace(go.Indicator(
    mode = "number+delta",
    value = int(type_grouped[1]),
#     title = {"text": f"High School Total"},
    domain = {'x': [0, 0.4], 'y': [0.75, 1]},
    align = "center",
    delta = {'reference': int(type_grouped_yesterday[1]), 'relative': True, 'position' : "right"}))

fig.update_layout(
    title={"text": "<span style='font-size:4em;color:black'>Total Cases</span>", 
           "yanchor":"middle",
           "xanchor":"center",
           "x":0.4,
           "y":0.85})

fig.update_layout(
    annotations=[
        dict(text="<span style='font-size:4em;color:black'>High School</span>",
             align="left",
            showarrow=False,
            x=0.72,
            y=0.95,),
        dict(text="<span style='font-size:4em;color:black'>Middle School</span>",
             align="left",
            showarrow=False,
            x=0.75,
            y=0.48,),
        dict(text="<span style='font-size:4em;color:black'>Elementary</span>",
             align="left",
            showarrow=False,
            x=0.7,
            y=0.05,)])

fig.show()

In [ ]:
df_sums = df_final[daily_columns].sum()
df_aggregated = df_sums.cumsum()

In [ ]:
df_aggregated

In [ ]:
import plotly.graph_objects as go
import numpy as np

title = 'Total Cases by Date'
colors = ['rgb(83,190,171)']

mode_size = [8]
line_size = [2]

fig = go.Figure()

fig.add_trace(go.Scatter(x=df_aggregated.index, y=df_aggregated, mode='lines',
    name=labels[0],
    line=dict(color=colors[0], width=2),
    connectgaps=True,
))

# endpoints
fig.add_trace(go.Scatter(
    x=[df_aggregated.index[-1]],
    y=[df_aggregated[-1]],
    mode='markers',
    marker=dict(color=colors[0], size=6)
))

fig.update_layout(
    xaxis=dict(
        showline=True,
        showgrid=False,
        showticklabels=True,
        linecolor='rgb(204, 204, 204)',
        linewidth=2,
        ticks='outside',
        tickfont=dict(
            family='Arial',
            size=12,
            color='rgb(82, 82, 82)',
        ),
    ),
    yaxis=dict(
        showgrid=False,
        zeroline=False,
        showline=True,
        showticklabels=True,
        tickfont=dict(
        family='Arial',
            size=12,
            color='rgb(82, 82, 82)')
    ),
    autosize=False,
    margin=dict(
        autoexpand=False,
        l=100,
        r=20,
        t=20,
    ),
    showlegend=False,
    plot_bgcolor='white'
)

fig.update_layout(annotations=[dict(xref='paper', yref='paper', x=0.5, y=0.9,
                              xanchor='center', yanchor='bottom',
                              text=title,
                              font=dict(size=26,
                                        color='rgb(82, 82, 82)'),
                              showarrow=False)])


fig.show()

In [ ]:
school_groups = df_final.groupby('school')
school_sums = school_groups['total_positive'].sum()
school_sums = school_sums.sort_values(ascending=False)
top_schools = school_sums[:10].sort_values(ascending=True)
top_schools

In [ ]:
import plotly.graph_objects as go

title = "Schools With Most Confirmed Cases"

fig = go.Figure(go.Bar(
            x=top_schools,
            y=top_schools.index,
            orientation='h',
            marker=dict(
                color='rgb(83,190,171)')))

fig.update_layout(
    xaxis=dict(
        showline=True,
        showgrid=False,
        showticklabels=True,
        linecolor='rgb(204, 204, 204)',
        linewidth=2,
        ticks='outside',
        tickfont=dict(
            family='Arial',
            size=12,
            color='rgb(82, 82, 82)',
        ),
    ),
    yaxis=dict(
        showgrid=False,
        zeroline=False,
        showline=True,
        showticklabels=True,
        tickfont=dict(
        family='Arial',
            size=12,
            color='rgb(82, 82, 82)')
    ),
    autosize=False,
    margin=dict(
        autoexpand=False,
        l=100,
        r=20,
        t=100,
    ),
    showlegend=False,
    plot_bgcolor='white'
)

fig.update_layout(annotations=[dict(xref='paper', yref='paper', x=0.5, y=1,
                              xanchor='center', yanchor='bottom',
                              text=title,
                              font=dict(size=26,
                                        color='rgb(82, 82, 82)'),
                              showarrow=False)])

fig.show()

# Plotly Bubble Map

In [119]:
df_final.head()

,school,type,Total Staff & Student,8/13/20,8/14/20,8/17/20,8/18/20,8/19/20,8/20/20,8/21/20,total_positive,percent_positive
0,Big Creek,ES,706.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
1,Brandywine,ES,1357.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
2,Brookwood,ES,1078.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
3,Chattahoochee,ES,850.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,2.0,0.235294
4,Chestatee,ES,1381.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.072411


In [120]:
columns = df_final.columns
daily_columns = columns[3:-2]
info_columns = columns[0:3]
df_cumulative = pd.merge(left=df_final[info_columns], right=df_final[daily_columns].cumsum(axis=1), left_index=True, right_index=True)

In [121]:
df_cumulative.head()

,school,type,Total Staff & Student,8/13/20,8/14/20,8/17/20,8/18/20,8/19/20,8/20/20,8/21/20
0,Big Creek,ES,706.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Brandywine,ES,1357.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Brookwood,ES,1078.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Chattahoochee,ES,850.0,0.0,0.0,0.0,2.0,2.0,2.0,2.0
4,Chestatee,ES,1381.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0


In [122]:
df_melted = pd.melt(df_cumulative, id_vars=df_cumulative.columns[0:3], value_vars=df_cumulative.columns[3:])
df_melted.rename(columns={'variable':'date', 'value':'total'}, inplace=True)

In [123]:
df_melted.tail(40)

,school,type,Total Staff & Student,date,total
233,West Forsyth,HS,2637.0,8/20/20,4.0
234,Big Creek,ES,706.0,8/21/20,0.0
235,Brandywine,ES,1357.0,8/21/20,0.0
236,Brookwood,ES,1078.0,8/21/20,0.0
237,Chattahoochee,ES,850.0,8/21/20,2.0
238,Chestatee,ES,1381.0,8/21/20,1.0
239,Coal Mountain,ES,821.0,8/21/20,2.0
240,Cumming,ES,1055.0,8/21/20,0.0
241,Daves Creek,ES,1285.0,8/21/20,0.0
242,Haw Creek,ES,1131.0,8/21/20,0.0


In [124]:
df_locations = pd.read_csv('school_locations.csv')

In [125]:
df_locations.drop(columns=df_locations.columns[0], inplace=True)
df_locations['latitude'] = pd.to_numeric(df_locations['geo'].str.split(', ', expand=True)[0])
df_locations['longitude'] = pd.to_numeric(df_locations['geo'].str.split(', ', expand=True)[1])
df_locations = df_locations.drop(columns='geo')
df_locations.head()

,school,type,Total Staff & Student,latitude,longitude
0,Big Creek,ES,706,34.113906,-84.177824
1,Brandywine,ES,1357,34.133500,-84.247383
2,Brookwood,ES,1078,34.101762,-84.184680
3,Chattahoochee,ES,850,34.239073,-84.086230
4,Chestatee,ES,1381,34.297109,-83.996014


In [195]:
df_geo = pd.merge(left=df_melted, right=df_locations, on=['school', 'type', 'Total Staff & Student'])

In [198]:
df_geo['Category'] = df_geo['type'].str.replace('ES', 'Elementary').replace('MS', 'Middle').replace('HS', 'High')

In [90]:
def create_hovertext(df):
    text_list = []
    for row in df.iterrows():
        text = f"<b>{row.school}</b><br><br>Total Cases: {row.total}<br>Total Staff & Student: {row['Total Staff & Student']}<br><extra></extra>"
        text_list.append(text)
    return text_list

In [91]:
df_geo['text'] = str("<b>" + str(df_geo['school']) + "</b><br><br>Total Cases: " + str(df_geo['total']) + "<br>Total Staff & Student: " + str(df_geo['Total Staff & Student']) + "<br><extra></extra>")

In [128]:
df_geo.tail()

,school,type,Total Staff & Student,date,total,latitude,longitude
268,West Forsyth,HS,2637.0,8/17/20,2.0,34.211249,-84.214213
269,West Forsyth,HS,2637.0,8/18/20,3.0,34.211249,-84.214213
270,West Forsyth,HS,2637.0,8/19/20,3.0,34.211249,-84.214213
271,West Forsyth,HS,2637.0,8/20/20,4.0,34.211249,-84.214213
272,West Forsyth,HS,2637.0,8/21/20,4.0,34.211249,-84.214213


In [129]:
fig = px.scatter_mapbox(df_geo, 
                        lat="latitude", 
                        lon="longitude", 
                        size=(df_geo.total + 0), 
                        hover_name=df_geo.school, 
                        text= df_geo.school,
                        color=df_geo.total,
                        color_continuous_scale=px.colors.sequential.Blugrn,
                        center={"lat":34.221749,
                                "lon":-84.135526},
                        zoom=9, 
                        height=300)
fig.update_layout(mapbox_style="open-street-map")
fig.update_layout(margin={"r":200,"t":0,"l":200,"b":0})
fig.show()

In [139]:
df_geo.tail(10)

,school,type,Total Staff & Student,date,total,latitude,longitude
263,South Forsyth,HS,2437.0,8/19/20,0.0,34.148412,-84.170303
264,South Forsyth,HS,2437.0,8/20/20,0.0,34.148412,-84.170303
265,South Forsyth,HS,2437.0,8/21/20,0.0,34.148412,-84.170303
266,West Forsyth,HS,2637.0,8/13/20,0.0,34.211249,-84.214213
267,West Forsyth,HS,2637.0,8/14/20,1.0,34.211249,-84.214213
268,West Forsyth,HS,2637.0,8/17/20,2.0,34.211249,-84.214213
269,West Forsyth,HS,2637.0,8/18/20,3.0,34.211249,-84.214213
270,West Forsyth,HS,2637.0,8/19/20,3.0,34.211249,-84.214213
271,West Forsyth,HS,2637.0,8/20/20,4.0,34.211249,-84.214213
272,West Forsyth,HS,2637.0,8/21/20,4.0,34.211249,-84.214213


In [138]:
df_geo.shape

(273, 7)

In [137]:
fig = px.scatter_mapbox(df_geo, 
                        lat=df_geo.latitude, 
                        lon=df_geo.longitude, 
                        animation_frame=df_geo.date,
                        animation_group=df_geo.school,
                        size=df_geo.total, 
                        hover_name=df_geo.school, 
                        text= df_geo.school,
                        color=df_geo.total,
                        color_continuous_scale=px.colors.sequential.Blugrn,
                        center={"lat":34.221749,
                                "lon":-84.135526},
                        zoom=9.8, 
                        height=600)
fig.update_layout(mapbox_style="open-street-map")
fig.update_layout(margin={"r":200,"t":0,"l":200,"b":0})
fig.show()

In [252]:
fig = px.scatter_mapbox(df_geo, 
                        lat="latitude", 
                        lon="longitude",
                        animation_frame = 'date', 
                        animation_group = 'school', 
                        color="Category",
                        size="total",
                        color_discrete_sequence=px.colors.qualitative.Prism,
                        center={"lat":34.221749,
                                "lon":-84.135526},
                        zoom=10,
                        height=700,
                        hover_name='school', 
                        hover_data = ['school', 'total', 'Total Staff & Student'])
fig.update_layout(mapbox_style="carto-positron")
fig.update_layout(margin={"r":0,"t":100,"l":0,"b":0},
                  title=dict(
                      text='Reported COVID-19 cases in Forsyth County Schools',
                      font=dict(
                          color='black',
                          size=26),
                          xanchor="center",
                          x=0.5))
fig.layout.sliders[0]['active'] = len(fig.frames) - 1
fig.update_traces(fig.frames[-1].data[-1])

fig.show()